In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/etkiketli.csv')

In [ ]:
list_to_drop =  []
for i in range(len(data["yorum"])):
    if type(data["yorum"].iloc[i]) is not str:
        list_to_drop.append(i)

data.drop(labels=list_to_drop,axis=0,inplace=True)

In [ ]:
def tokenize_the_data_from_pandas(dataframe,column_name): 
  
    import string
    for sent in range(len(data[column_name])):
        example_sentence = data[column_name].iloc[sent]
        new_sentence = example_sentence.translate(str.maketrans("","",string.punctuation))
        data[column_name].iloc[sent] = new_sentence
    tokens = {}
    for sent in range(len(data[column_name])):
        example_sentence = data[column_name].iloc[sent]
        values = example_sentence.split(" ")
        for word in values:
            tokens[word] = 0
         
    names = list(tokens.keys())
    for num in range(len(names)):
        tokens[names[num]] = num+1
        
    return tokens

In [ ]:
tokens = tokenize_the_data_from_pandas(data,column_name="yorum")

In [ ]:
len(tokens.keys())

In [ ]:
max_length = 0
for sentence in data["yorum"]:
    new_sentence = sentence.translate(str.maketrans("","",string.punctuation))
    length = len(new_sentence.split(" "))
    if length > max_length:
        max_length = length

In [ ]:
X = []
y = []
for sent in range(len(data["yorum"])):
    example_sentence = data["yorum"].iloc[sent]
    values = example_sentence.split(" ")
    for word in range(len(values)):
        values[word] = tokens[values[word]]
    X.append(values)
    y.append(data["Analysis"].iloc[sent])
    

from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = max_length
padded_docs = pad_sequences(X, maxlen=max_length, padding='post')

türler = list(data["Analysis"].unique())

new_y = []
for tür in range(len(data["Analysis"])):
    st_liste = [0,0,0]
    itham = data["Analysis"].iloc[tür]
    st_liste[türler.index(itham)] += 1
    new_y.append(st_liste)
X_ar = np.array(padded_docs)
y_ar = np.array(new_y)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_ar,y_ar,test_size=0.1,random_state = 5)

In [ ]:
X_train.shape

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(45400,400,input_length=max_length))
model.add(tf.keras.layers.Conv1D(16,10,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Conv1D(32,7,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Conv1D(64,5,activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(2))
model.add(tf.keras.layers.Dropout(0.2))

In [ ]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer = "adam",loss="categorical_crossentropy",metrics=["accuracy"])
history=model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test, batch_size=200,verbose=0)
    
y_pred_bool = np.argmax(y_pred,axis=1)
y_test_bol = np.argmax(y_test, axis=1)
class_names = ["Negatif","Neutar","Positive"]

print(classification_report(y_test_bol, y_pred_bool,target_names=class_names))

In [ ]:
import sklearn.metrics as metrics
confusion_matrix = metrics.confusion_matrix(y_test_bol, y_pred_bool)
confusion_matrix

In [ ]:
sensitivity = confusion_matrix[0,0]/(confusion_matrix[0,0]+confusion_matrix[0,1])
print('Sensitivity : ', sensitivity )

In [ ]:
specificity = confusion_matrix[1,1]/(confusion_matrix[1,0]+confusion_matrix[1,1])
print('Specificity : ', specificity)

In [ ]:
from sklearn.metrics import precision_score
precisionScore_sklearn_microavg = precision_score(y_test_bol, y_pred_bool, average='micro')
print('Precision :',precisionScore_sklearn_microavg)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['training data', 'validation data'], loc = 'upper right')

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylim(0, 1)
plt.title('model accuracy (no dropout)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.subplot(2,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss (no dropout)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.ylim([0,2])
plt.show()